In [ ]:
!pip install -q -U transformers==4.37.2
!pip install --upgrade "transformers>=4.47.0" "peft>=0.9.0"
!pip install -q  accelerate==0.26.0
!pip install --upgrade "accelerate>=0.27.0"
!pip install --prefer-binary bitsandbytes
!pip install -U bitsandbytes
!python -m bitsandbytes
!nvidia-smi


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 99.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.2.0 requires transformers<6.0.0,>=4.41.0, but you have transformers 4.37.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 144.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 119.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing in

In [ ]:
from google.colab import drive
import sys,os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
drive.mount('/content/drive')




Mounted at /content/drive


In [ ]:
modules_dir = '/content/drive/MyDrive/lllava_num_pec_imprv_github/from_google_drive/Modules'
config_path = '/content/drive/MyDrive/lllava_num_pec_imprv_github/downloaded_from_vm_instance/train_config_new.json'
sys.path.append(modules_dir)

from llava_utils import *
from data_utils import *
from vit_finetuning_utils import *


dataset_config, training_config, lora_config = load_config(config_path)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/674 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

CONFIGURATION SUMMARY

📊 DATASET & TRAINING:
  Train Dataset: CCNL2
  Val Dataset: CCNL2
  Test Dataset: CCNL1
  Learning Rate: 1e-05
  Total Epochs: 8
  Batch Size: 3

💾 SAVE PATHS:
  Predictions Save Path: /home/timur_oner/experiments/adapters_buffer
  Adapter Save Path: /home/timur_oner/experiments/adapters_buffer
  Prompt Inference: USER: <image>
How many objects are there in the image? Respond only with the number.
ASSISTANT:

🔢 DATA SPLITS:
  Train Numbers: [2, 4, 6, 8, 10] (Total: 5)
  Val Numbers: [2, 4, 6, 8, 10] (Total: 5)
  Test Numbers: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] (Total: 10)

🧩 DATA AUGMENTATION:
  Augmentation Enabled: no
  Augmentation Factor: 2
  Extra Basic Aug: True

🎯 OBJECT CATEGORIES:
  Train Objects: ['apples', 'cups'] (Total: 2)
  Val Objects: ['buttons'] (Total: 1)
  Test Objects: ['triangles', 'butterflies'] (Total: 2)

🔧 LORA CONFIGURATION:
  Rank (r): 16
  Alpha: 16
  Dropout: 0.05
  Bias: none
  Task Type: CAUSAL_LM
  Target Modules: multi_modal_projecto

In [ ]:
def prepare_datasets_and_dataloaders(dataset_config, processor, batch_size, custom_collate_fn):
    DATASET_PATHS = {
        "CCNL1": '/content/drive/MyDrive/Modality Gap and Numerosity Thesis/models/images',
        "CCNL2": '/content/drive/MyDrive/Modality Gap and Numerosity Thesis/CCNL2',
        "CLEVR": "/home/timur_oner/datasets/CLEVR",
        "CHLDBOOK": '/content/drive/MyDrive/Modality Gap and Numerosity Thesis/childbook',
    }

    def resolve_dataset_path(name: str) -> str:
        if name not in DATASET_PATHS:
            raise ValueError(f"Unknown dataset: {name}")
        return DATASET_PATHS[name]

    train_dataset_path = resolve_dataset_path(dataset_config.train_dataset_name)
    val_dataset_path   = resolve_dataset_path(dataset_config.val_dataset_name)
    test_dataset_path  = resolve_dataset_path(dataset_config.test_dataset_name)

    train_dataset = ImageTextDataset(
        image_base_path=train_dataset_path,
        dataset=dataset_config.train_dataset_name,
        processor=processor,
        nums=dataset_config.train_nums,
        categories=dataset_config.train_objs,
        undersamp_ratio=1,
        sample_idx_range=range(0, 40)
    )

    val_dataset = ImageTextDataset(
        image_base_path=val_dataset_path,
        dataset=dataset_config.val_dataset_name,
        processor=processor,
        nums=dataset_config.val_nums,
        categories=dataset_config.val_objs,
        undersamp_ratio=1,
        sample_idx_range=range(40, 50)
    )

    test_dataset = ImageTextDataset(
        image_base_path=test_dataset_path,
        dataset=dataset_config.test_dataset_name,
        processor=processor,
        nums=dataset_config.test_nums,
        categories=dataset_config.test_objs,
        test_mode=True,
        undersamp_ratio=0.1
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True,
        collate_fn=custom_collate_fn,
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True,
        collate_fn=custom_collate_fn,
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True,
        collate_fn=custom_collate_fn,
    )

    return train_loader, val_loader, test_loader,train_dataset,val_dataset,test_dataset




In [ ]:
tokenizer, processor, peft_model, device = prepare_peft_model(lora_config, training_config)
train_loader, val_loader, test_loader,train_dataset,val_dataset,test_dataset = prepare_datasets_and_dataloaders(dataset_config, processor, training_config.batch_size, custom_collate_fn)



print('Sanity Checking test dataset...')
sanity_check_dataset(test_dataset,peft_model,tokenizer,device)





# The optimizer and the training

optimizer = torch.optim.Adam([param for param in peft_model.parameters() if param.requires_grad],lr=training_config.learning_rate,
                             betas=(0.9, 0.98), eps=1e-5, weight_decay=training_config.weight_decay)
trained_model, train_losses, val_losses = train_model(
    model=peft_model,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    optimizer=optimizer,
    device=device,
    epochs=training_config.total_epochs,
    early_stop_tolerance=training_config.early_stop_tolerance,
    early_stop_perc_thr=training_config.early_stop_trig,
    use_mixed_precision=True,


)


config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

Sanity Checking test dataset...

=== Dataset Sanity Check ===
Sample index: 67/100
Image path: /content/drive/MyDrive/Modality Gap and Numerosity Thesis/models/images/1_butterflies_558.png
Original numerical label: 1

Pixel values shape: torch.Size([3, 336, 336])
Pixel values: min=-1.792, max=2.146, mean=2.025

Input IDs shape: torch.Size([605])
Input IDs: tensor([    1,  3148,  1001, 29901, 29871, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000,
        32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000])
Decoded input: USER:  
How many objects are there in the image? Respond only with the number.
ASSISTANT: 

Attention mask shape: torch.Size([605])
Attention mask sum: 605

Labels shape: torch.Size([605])
Labels (non -100): tensor([], dtype=torch.int64)
Decoded labels: 

=== Testing with model ===
Model output: USER:  
How many objects are there in the image? Respond only with the number.
ASSISTANT: 1


/content/drive/MyDrive/lllava_num_pec_imprv_github/from_google_drive/Modules/llava_utils.py:417: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler() if use_mixed_precision else None
Epoch 1/8:   0%|          | 0/134 [00:00<?, ?it/s]/content/drive/MyDrive/lllava_num_pec_imprv_github/from_google_drive/Modules/llava_utils.py:443: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on th

In [ ]:
from torch.nn.utils.rnn import pad_sequence
trained_model, train_losses, val_losses = train_model(
    model=peft_model,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    optimizer=optimizer,
    device=device,
    epochs=total_epochs,
    early_stop_tolerance=early_stop_tolerance,
    early_stop_perc_thr=early_stop_trig,
    use_mixed_precision=True,


)


In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()